In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../inc')


In [3]:
# Load raw lodging list
df = pd.read_csv('../DABA/서울시 관광숙박업 인허가 데이터.csv', encoding='cp949')
#df = pd.read_csv('../DABA/서울시 숙박업 인허가 정보.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 61 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   개방자치단체코드    610 non-null    int64  
 1   관리번호        610 non-null    object 
 2   인허가일자       610 non-null    object 
 3   인허가취소일자     1 non-null      float64
 4   영업상태코드      610 non-null    int64  
 5   영업상태명       610 non-null    object 
 6   상세영업상태코드    610 non-null    int64  
 7   상세영업상태명     610 non-null    object 
 8   폐업일자        136 non-null    object 
 9   휴업시작일자      9 non-null      object 
 10  휴업종료일자      9 non-null      object 
 11  재개업일자       0 non-null      float64
 12  전화번호        508 non-null    object 
 13  소재지면적       0 non-null      float64
 14  소재지우편번호     236 non-null    object 
 15  지번주소        610 non-null    object 
 16  도로명주소       602 non-null    object 
 17  도로명우편번호     516 non-null    float64
 18  사업장명        610 non-null    object 
 19  최종수정일자      610 non-null    o

In [9]:
# select item
sel_colum = [2,3,4,8,9,10,11,15,16,17,18,21,23,24,32,35,46,59,60]
sel_df = df.iloc[:,sel_colum]
sel_df.info()

if pd.notnull(sel_df.지번주소[137]): print('111')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   인허가일자     610 non-null    object 
 1   인허가취소일자   1 non-null      float64
 2   영업상태코드    610 non-null    int64  
 3   폐업일자      136 non-null    object 
 4   휴업시작일자    9 non-null      object 
 5   휴업종료일자    9 non-null      object 
 6   재개업일자     0 non-null      float64
 7   지번주소      610 non-null    object 
 8   도로명주소     602 non-null    object 
 9   도로명우편번호   516 non-null    float64
 10  사업장명      610 non-null    object 
 11  데이터갱신일자   610 non-null    object 
 12  좌표정보(X)   603 non-null    float64
 13  좌표정보(Y)   603 non-null    float64
 14  건물용도명     209 non-null    object 
 15  객실수       307 non-null    float64
 16  시설면적      358 non-null    float64
 17  시설규모      358 non-null    float64
 18  관광숙박업상세명  319 non-null    object 
dtypes: float64(8), int64(1), object(10)
memory usage: 90.7+ KB
111


In [13]:
# .. 카카오API로 위경도 삽입, pyproj로 시도하였으나 값이 약간 다름

import requests # 온라인으로 보내거나 받아올 때 사용, 파이썬으로 웹사이트 연동할 때
from urllib.parse import quote # url에서 한글 코드를 변환
import numpy as np


# API 키 가져오기
filename = '/Users/mankikim/Desktop/프로젝트/더조은/API/kakao_API.txt'
with open(filename) as f:
    api_key = f.read().strip()
    
len(api_key)  # 32글자가 되야됨, 파일키 정상 문자열개수
     
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lon_list, err_list = [], [], []

for i in sel_df.index:
#for i in [37]:
    print(f'{i} , {sel_df.사업장명[i]}')

    addr = np.nan
    
    if pd.notnull(sel_df.지번주소[i]): addr = sel_df.지번주소[i]    # 관광데이터는 지번주소가 무조건 있음
    #if pd.notnull(sel_df.도로명주소[i]): addr = sel_df.도로명주소[i]
    #if pd.isna(addr): print( f"{i}번째 {sel_df.사업장명[i]}은 주소가 아무것도 없습니다, 패스")

    temp = quote(addr)

    url = f'{search_url}?query={temp}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()

    print(result)

    try:
        lon_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        lon_list.append(np.NAN)
        lat_list.append(np.NAN)
        err_list.append((i,sel_df.지번주소[i]))

    
sel_df['위도'] = lat_list
sel_df['경도'] = lon_list

sel_df.head()

0 , 호텔 데님(Hotel Denim)
{'documents': [{'address': {'address_name': '서울 강남구 개포동 1229-14', 'b_code': '1168010300', 'h_code': '1168069000', 'main_address_no': '1229', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_h_name': '개포4동', 'region_3depth_name': '개포동', 'sub_address_no': '14', 'x': '127.046615783737', 'y': '37.4761898374266'}, 'address_name': '서울 강남구 개포동 1229-14', 'address_type': 'REGION_ADDR', 'road_address': {'address_name': '서울 강남구 논현로 66', 'building_name': '데님관광호텔', 'main_building_no': '66', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_name': '개포동', 'road_name': '논현로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.046653660142', 'y': '37.4761953186367', 'zone_no': '06307'}, 'x': '127.046615783737', 'y': '37.4761898374266'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
1 , 티마크그랜드호텔명동
{'documents': [{'address': {'address_name': '서울 중구 회현동1가 194-15', 'b_code': '1114012100', 'h_cod

/var/folders/wj/6fqsj4yn655_2rsnl7jt1yyh0000gn/T/ipykernel_6892/1991862524.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['위도'] = lat_list
/var/folders/wj/6fqsj4yn655_2rsnl7jt1yyh0000gn/T/ipykernel_6892/1991862524.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['경도'] = lon_list


,인허가일자,인허가취소일자,영업상태코드,폐업일자,휴업시작일자,휴업종료일자,재개업일자,지번주소,도로명주소,도로명우편번호,...,데이터갱신일자,좌표정보(X),좌표정보(Y),건물용도명,객실수,시설면적,시설규모,관광숙박업상세명,위도,경도
0,2015-02-03,NaN,1,NaN,NaN,NaN,NaN,서울특별시 강남구 개포동 1229-14,서울특별시 강남구 논현로 66 (개포동),6307.0,...,2022-12-02 22:09:00.0,204053.596301,441557.179656,NaN,NaN,NaN,NaN,NaN,37.476190,127.046616
1,2016-05-17,NaN,3,2023-02-28,NaN,NaN,NaN,서울특별시 중구 회현동1가 194-15,"서울특별시 중구 퇴계로 52 (회현동1가, 인송빌딩)",4634.0,...,2022-12-03 00:03:00.0,198043.635511,450686.849957,NaN,NaN,NaN,NaN,NaN,37.558454,126.978646
2,2021-04-06,NaN,1,NaN,NaN,NaN,NaN,서울특별시 금천구 독산동 335-21,서울특별시 금천구 가산로3길 103 (독산동),8526.0,...,2022-12-02 23:02:00.0,190183.499829,441092.525925,NaN,NaN,NaN,NaN,NaN,37.471959,126.889817
3,2014-07-22,NaN,1,NaN,NaN,NaN,NaN,서울특별시 중구 충무로1가 24-30,서울특별시 중구 퇴계로 107 (충무로1가),4536.0,...,2022-12-02 23:05:00.0,198484.096466,450963.971753,NaN,NaN,NaN,NaN,NaN,37.560953,126.983631
4,2016-03-29,NaN,2,NaN,2023-01-01,2023-12-31,NaN,서울특별시 강남구 논현동 205-8,"서울특별시 강남구 봉은사로 143 (논현동, 운현오피스텔)",6122.0,...,2022-12-03 23:02:00.0,202596.666692,444917.728061,NaN,NaN,NaN,NaN,NaN,37.506474,127.030156


In [14]:
print(err_list)
print(len(err_list))
sel_df.info()


[(37, '서울특별시 강남구 논현동 6 6-1'), (44, '서울특별시 강서구 방화동 886 외 6필지'), (73, '서울특별시 관악구 남현동 1061-2 3~7층'), (89, '서울특별시 중구 순화동 214-0 외 1필지'), (94, '서울특별시 성동구 도선동 58-0번지 '), (95, '서울특별시 강북구 수유동 174-32 외1필지(174-19)'), (117, '서울특별시 마포구 합정동 381-17번지 '), (150, '서울특별시 종로구 수송동 51-8 '), (175, '서울특별시 종로구 당주동 29 '), (194, '서울특별시 중구 을지로6가 17-2번지 케레스타 빌딩 18층~20층 '), (212, '서울특별시 중구 북창동 93-99 '), (227, '서울특별시 중구 회현동1가 194-19 '), (233, '서울특별시 중구 광희동2가 360번지 3-7층 '), (238, '서울특별시 중구 광희동2가 43-1 3~6층 '), (266, '서울특별시 중구 태평로1가 51-1 '), (274, '서울특별시 중구 순화동 151번지 외 1필지 C동 '), (280, '서울특별시 용산구 이태원동 119-24번지 119-24,25,108-9,123-10~15,116-1,116-5,112-7,112-15 '), (281, '서울특별시 용산구 이태원동 34-69 36-69, 34-159, 44-64'), (294, '서울특별시 성동구 도선동 72번지 외 2필지(73, 74번지) '), (299, '서울특별시 광진구 광장동 산 21-0번지 '), (315, '서울특별시 광진구 광장동 산 21-0'), (332, '서울특별시 성북구 동선동1가 92-20 외 3필지(92-21, 92-26, 92-27)'), (356, '서울특별시 서대문구 창천동 31-92 8,9,10층'), (401, '서울특별시 강서구 화곡동 918-23번지 외 2필지 '), (447, '서울특별시 영등포구 당산동3가 136번지 외 1필지 '), (574, '서울특별시 강남구 논현동

In [19]:
# 사업장명 컬럼 맨 앞으로 
sel_df2 = sel_df[['사업장명','관광숙박업상세명','건물용도명','지번주소','도로명주소','위도','경도','객실수', '시설면적','시설규모','인허가일자','인허가취소일자','영업상태코드','폐업일자','휴업시작일자','휴업종료일자','재개업일자','데이터갱신일자']]
sel_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   사업장명      610 non-null    object 
 1   관광숙박업상세명  319 non-null    object 
 2   건물용도명     209 non-null    object 
 3   지번주소      610 non-null    object 
 4   도로명주소     602 non-null    object 
 5   위도        583 non-null    float64
 6   경도        583 non-null    float64
 7   객실수       307 non-null    float64
 8   시설면적      358 non-null    float64
 9   시설규모      358 non-null    float64
 10  인허가일자     610 non-null    object 
 11  인허가취소일자   1 non-null      float64
 12  영업상태코드    610 non-null    int64  
 13  폐업일자      136 non-null    object 
 14  휴업시작일자    9 non-null      object 
 15  휴업종료일자    9 non-null      object 
 16  재개업일자     0 non-null      float64
 17  데이터갱신일자   610 non-null    object 
dtypes: float64(7), int64(1), object(10)
memory usage: 85.9+ KB


In [20]:
sel_df2.to_csv("../DAIN/crw_small_list.txt")

In [21]:
err_df = pd.DataFrame(err_list, columns=['index','위경도 missing'])
err_df.to_csv("../DAIN/err_small_list.txt")